## Segmentation in Pinot

Table contents in Pinot are expected to grow infinitely and thus need to be distributed across multiple nodes. The dataset is split into segments, which are comparable to shards/partitions in classical RDBMS. Segmentation is done in a time-based fashion, meaning that rows in a given segment will be timewisely close to each other.
Segments store all columns of a table and organize data in columnar orientation for high encoding efficiency and optional pre-aggregation of metrics. In addition to values, segments store indices and other lookup-related data structures like dictionaries. By default values are stored using dictionary encoding, meaning that values are represented as dictionary IDs that reference a corresponding dictionary entry. This way, values can be stored with the minimum number of bits required, which depends on the cardinality of the column. 

In [77]:
# all imports
import requests
import json
import io
import re
import pandas as pd
from kafka import KafkaConsumer

In [171]:
# some helpers
def query_sql(query):
    print("query: " + query)
    return requests.post('http://pinot-broker.pinot:8099/query/sql', json={
        "sql" : query
    }).json()

def query_sql_dataframe(query):
    result = query_sql(query)
    return pd.DataFrame(columns=result['resultTable']['dataSchema']['columnNames'], data=result['resultTable']['rows'])

ordinal_pattern = re.compile(r'__[0-9]+__([0-9]+)__')
def sort_by_ascending_ordinal(segments):
    segments.sort(key=lambda L: (int(ordinal_pattern.search(L).group(1)), L))

In [79]:
consumer = KafkaConsumer(group_id='test', bootstrap_servers=['pinot-kafka.pinot:9092'])
consumer.topics()

{'my-trips_gendata3',
 'trips',
 'trips_gendata',
 'trips_gendata2',
 'trips_gendata3'}

In [80]:
requests.get('http://pinot-controller.pinot:9000/schemas/trips').json()

{'schemaName': 'trips',
 'dimensionFieldSpecs': [{'name': 'rider_name',
   'dataType': 'STRING',
   'defaultNullValue': ''},
  {'name': 'driver_name', 'dataType': 'STRING', 'defaultNullValue': ''},
  {'name': 'license_plate', 'dataType': 'STRING', 'defaultNullValue': ''},
  {'name': 'start_location', 'dataType': 'STRING', 'defaultNullValue': ''},
  {'name': 'start_zip_code', 'dataType': 'STRING', 'defaultNullValue': ''},
  {'name': 'end_location', 'dataType': 'STRING', 'defaultNullValue': ''},
  {'name': 'end_zip_code', 'dataType': 'STRING', 'defaultNullValue': ''},
  {'name': 'rider_is_premium', 'dataType': 'INT', 'defaultNullValue': 0}],
 'metricFieldSpecs': [{'name': 'count',
   'dataType': 'LONG',
   'defaultNullValue': 1},
  {'name': 'payment_amount', 'dataType': 'FLOAT'},
  {'name': 'payment_tip_amount', 'dataType': 'FLOAT'},
  {'name': 'trip_wait_time_millis', 'dataType': 'LONG'},
  {'name': 'rider_rating', 'dataType': 'INT'},
  {'name': 'driver_rating', 'dataType': 'INT'}],
 'd

In [81]:
table_config = {
  "tableName": "",
  "tableType": "REALTIME",
  "segmentsConfig": {
    "timeColumnName": "trip_start_time_millis",
    "timeType": "MILLISECONDS",
    "retentionTimeUnit": "DAYS",
    "retentionTimeValue": "60",
    "schemaName": "trips",
    "replication": "1",
    "replicasPerPartition": "1"
  },
  "tenants": {},
  "tableIndexConfig": {
    "loadMode": "MMAP",
    "invertedIndexColumns": [
        "rider_name",
        "driver_name",
        "start_location",
        "end_location"
    ],
    "streamConfigs": {
      "streamType": "kafka",
      "stream.kafka.consumer.type": "simple",
      "stream.kafka.topic.name": "trips",
      "stream.kafka.decoder.class.name": "org.apache.pinot.plugin.stream.kafka.KafkaJSONMessageDecoder",
      "stream.kafka.consumer.factory.class.name": "org.apache.pinot.plugin.stream.kafka20.KafkaConsumerFactory",
      "stream.kafka.zk.broker.url": "pinot-kafka-zookeeper:2181",
      "stream.kafka.broker.list": "pinot-kafka:9092",
      "realtime.segment.flush.threshold.time": "12h",
      "realtime.segment.flush.threshold.size": "5000",
      "stream.kafka.consumer.prop.auto.offset.reset": "smallest"
    }
  },
  "metadata": {
    "customConfigs": {}
  }
}

table_config["tableName"] = "trips_segmentation_1"
print(requests.post('http://pinot-controller.pinot:9000/tables', json=table_config).json())

table_config["tableName"] = "trips_segmentation_2"
table_config["segmentsConfig"]["replication"] = "3"
table_config["segmentsConfig"]["replicasPerPartition"] = "3"
table_config["tableIndexConfig"]["streamConfigs"]["realtime.segment.flush.threshold.size"] = "10000"
print(requests.post('http://pinot-controller.pinot:9000/tables', json=table_config).json())

{'status': 'Table trips_segmentation_1_REALTIME succesfully added'}
{'status': 'Table trips_segmentation_2_REALTIME succesfully added'}


In [82]:
requests.get('http://pinot-controller.pinot:9000/tables/trips_segmentation_1/instances').json()

{'tableName': 'trips_segmentation_1',
 'brokers': [{'tableType': 'realtime',
   'instances': ['Broker_pinot-broker-0.pinot-broker-headless.pinot.svc.cluster.local_8099']}],
 'server': [{'tableType': 'realtime',
   'instances': ['Server_pinot-server-2.pinot-server-headless.pinot.svc.cluster.local_8098',
    'Server_pinot-server-0.pinot-server-headless.pinot.svc.cluster.local_8098',
    'Server_pinot-server-1.pinot-server-headless.pinot.svc.cluster.local_8098']}]}

In [83]:
requests.get('http://pinot-controller.pinot:9000/tables/trips_segmentation_2/instances').json()

{'tableName': 'trips_segmentation_2',
 'brokers': [{'tableType': 'realtime',
   'instances': ['Broker_pinot-broker-0.pinot-broker-headless.pinot.svc.cluster.local_8099']}],
 'server': [{'tableType': 'realtime',
   'instances': ['Server_pinot-server-2.pinot-server-headless.pinot.svc.cluster.local_8098',
    'Server_pinot-server-0.pinot-server-headless.pinot.svc.cluster.local_8098',
    'Server_pinot-server-1.pinot-server-headless.pinot.svc.cluster.local_8098']}]}

In [343]:
response = requests.get('http://pinot-controller.pinot:9000/segments/trips_segmentation_1').json()
segments_1 = response[0]['REALTIME']
sort_by_ascending_ordinal(segments_1)
pd.DataFrame(segments_1, columns=['trips_segmentation_1'])

,trips_segmentation_1
0,trips_segmentation_1__0__0__20210409T2006Z
1,trips_segmentation_1__0__1__20210409T2007Z
2,trips_segmentation_1__0__2__20210409T2007Z
3,trips_segmentation_1__0__3__20210409T2007Z
4,trips_segmentation_1__0__4__20210409T2007Z
5,trips_segmentation_1__0__5__20210409T2007Z
6,trips_segmentation_1__0__6__20210409T2007Z
7,trips_segmentation_1__0__7__20210409T2007Z
8,trips_segmentation_1__0__8__20210409T2007Z
9,trips_segmentation_1__0__9__20210409T2007Z


In [104]:
response = requests.get('http://pinot-controller.pinot:9000/segments/trips_segmentation_2').json()
segments_2 = response[0]['REALTIME']
sort_by_ascending_ordinal(segments_2)
pd.DataFrame(segments_2, columns=['trips_segmentation_2'])

,trips_segmentation_2
0,trips_segmentation_2__0__0__20210409T2006Z
1,trips_segmentation_2__0__1__20210409T2007Z
2,trips_segmentation_2__0__2__20210409T2007Z
3,trips_segmentation_2__0__3__20210409T2007Z
4,trips_segmentation_2__0__4__20210409T2007Z
5,trips_segmentation_2__0__5__20210409T2007Z
6,trips_segmentation_2__0__6__20210409T2007Z
7,trips_segmentation_2__0__7__20210409T2007Z
8,trips_segmentation_2__0__8__20210409T2007Z
9,trips_segmentation_2__0__9__20210409T2007Z


In [105]:
segment_metadata_1 = {}

for segment in segments_1:
    segment_metadata_1[segment] = requests.get(f'http://pinot-controller.pinot:9000/segments/trips_segmentation_1/{segment}/metadata').json()

pd.DataFrame(segment_metadata_1)

,trips_segmentation_1__0__0__20210409T2006Z,trips_segmentation_1__0__1__20210409T2007Z,trips_segmentation_1__0__2__20210409T2007Z,trips_segmentation_1__0__3__20210409T2007Z,trips_segmentation_1__0__4__20210409T2007Z,trips_segmentation_1__0__5__20210409T2007Z,trips_segmentation_1__0__6__20210409T2007Z,trips_segmentation_1__0__7__20210409T2007Z,trips_segmentation_1__0__8__20210409T2007Z,trips_segmentation_1__0__9__20210409T2007Z,...,trips_segmentation_1__0__12__20210409T2007Z,trips_segmentation_1__0__13__20210409T2007Z,trips_segmentation_1__0__14__20210409T2007Z,trips_segmentation_1__0__15__20210409T2007Z,trips_segmentation_1__0__16__20210409T2007Z,trips_segmentation_1__0__17__20210409T2007Z,trips_segmentation_1__0__18__20210409T2007Z,trips_segmentation_1__0__19__20210409T2007Z,trips_segmentation_1__0__20__20210409T2007Z,trips_segmentation_1__0__21__20210409T2007Z
segment.realtime.endOffset,5000,10000,15000,20000,25000,30000,35000,40000,45000,50000,...,65000,70000,75000,80000,85000,90000,95000,100000,105000,9223372036854775807
segment.time.unit,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,...,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,None
segment.start.time,1617998917336,1618003880310,1618008889895,1618013900421,1618018879391,1618023935633,1618028898841,1618033868041,1618038902861,1618043959620,...,1618058923086,1618063956205,1618069002192,1618073898108,1618078867162,1618083889813,1618088846163,1618093876820,1618098961982,-1
segment.flush.threshold.size,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,...,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
segment.realtime.startOffset,0,5000,10000,15000,20000,25000,30000,35000,40000,45000,...,60000,65000,70000,75000,80000,85000,90000,95000,100000,105000
segment.end.time,1618007383025,1618012296121,1618017347537,1618022370669,1618027316643,1618032375175,1618037406714,1618042324496,1618047422458,1618052401879,...,1618067335919,1618072384205,1618077355098,1618082401811,1618087325128,1618092395601,1618097377066,1618102426062,1618107355214,-1
segment.total.docs,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,...,5000,5000,5000,5000,5000,5000,5000,5000,5000,-1
segment.table.name,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,...,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME,trips_segmentation_1_REALTIME
segment.realtime.numReplicas,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
segment.creation.time,1617998799391,1617998833590,1617998835430,1617998837264,1617998840189,1617998841735,1617998844131,1617998845713,1617998848157,1617998850289,...,1617998856919,1617998859162,1617998861218,1617998863105,1617998865506,1617998867388,1617998869827,1617998871608,1617998874151,1617998875798


In [192]:
segment_metadata_2 = {}

for segment in segments_2:
    segment_metadata_2[segment] = requests.get(f'http://pinot-controller.pinot:9000/segments/trips_segmentation_2/{segment}/metadata').json()

pd.DataFrame(segment_metadata_2)

,trips_segmentation_2__0__0__20210409T2006Z,trips_segmentation_2__0__1__20210409T2007Z,trips_segmentation_2__0__2__20210409T2007Z,trips_segmentation_2__0__3__20210409T2007Z,trips_segmentation_2__0__4__20210409T2007Z,trips_segmentation_2__0__5__20210409T2007Z,trips_segmentation_2__0__6__20210409T2007Z,trips_segmentation_2__0__7__20210409T2007Z,trips_segmentation_2__0__8__20210409T2007Z,trips_segmentation_2__0__9__20210409T2007Z,trips_segmentation_2__0__10__20210409T2007Z
segment.realtime.endOffset,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,9223372036854775807
segment.time.unit,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,MILLISECONDS,None
segment.start.time,1617998917336,1618008889895,1618018879391,1618028898841,1618038902861,1618048909607,1618058923086,1618069002192,1618078867162,1618088846163,-1
segment.flush.threshold.size,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000
segment.realtime.startOffset,0,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000
segment.end.time,1618012296121,1618022370669,1618032375175,1618042324496,1618052401879,1618062349418,1618072384205,1618082401811,1618092395601,1618102426062,-1
segment.total.docs,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,-1
segment.table.name,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME,trips_segmentation_2_REALTIME
segment.realtime.numReplicas,3,3,3,3,3,3,3,3,3,3,3
segment.creation.time,1617998799586,1617998835639,1617998840400,1617998844304,1617998848237,1617998852588,1617998857344,1617998861484,1617998865748,1617998869976,1617998874271


In [402]:
# get data from roughly the first 6 segments
start_time_ms_of_segment_7 = segment_metadata_1[segments_1[6]]["segment.start.time"]
query_for_trips_segmentation_1 = f"select count(*) from trips_segmentation_1 where trip_start_time_millis < {start_time_ms_of_segment_7}"
query_sql(query_for_trips_segmentation_1)

query: select count(*) from trips_segmentation_1 where trip_start_time_millis < 1618028898841


{'resultTable': {'dataSchema': {'columnDataTypes': ['LONG'],
   'columnNames': ['count(*)']},
  'rows': [[28258]]},
 'exceptions': [],
 'numServersQueried': 1,
 'numServersResponded': 1,
 'numSegmentsQueried': 22,
 'numSegmentsProcessed': 6,
 'numSegmentsMatched': 6,
 'numConsumingSegmentsQueried': 1,
 'numDocsScanned': 28258,
 'numEntriesScannedInFilter': 5000,
 'numEntriesScannedPostFilter': 0,
 'numGroupsLimitReached': False,
 'totalDocs': 106318,
 'timeUsedMs': 4,
 'segmentStatistics': [],
 'traceInfo': {},
 'minConsumingFreshnessTimeMs': 1617998876089}

In [405]:
# get data from roughly the first 3 segments
start_time_ms_of_segment_4 = segment_metadata_2[segments_2[3]]["segment.start.time"]
query_for_trips_segmentation_2 = f"select count(*) from trips_segmentation_2 where trip_start_time_millis < {start_time_ms_of_segment_4}"
query_sql(query_for_trips_segmentation_2)

query: select count(*) from trips_segmentation_2 where trip_start_time_millis < 1618028898841


{'resultTable': {'dataSchema': {'columnDataTypes': ['LONG'],
   'columnNames': ['count(*)']},
  'rows': [[28258]]},
 'exceptions': [],
 'numServersQueried': 3,
 'numServersResponded': 3,
 'numSegmentsQueried': 11,
 'numSegmentsProcessed': 3,
 'numSegmentsMatched': 3,
 'numConsumingSegmentsQueried': 1,
 'numDocsScanned': 28258,
 'numEntriesScannedInFilter': 10000,
 'numEntriesScannedPostFilter': 0,
 'numGroupsLimitReached': False,
 'totalDocs': 106318,
 'timeUsedMs': 7,
 'segmentStatistics': [],
 'traceInfo': {},
 'minConsumingFreshnessTimeMs': 1617998877198}

## Query Routing / Processing

Brokers are responsible for maintaining routing tables, which contain mappings between segments of a table and servers where they are hosted on. This allows brokers to efficiently scatter received queries across servers.

In [383]:
# some helpers
server_name_regex = re.compile('pinot-server-[0-9]+')

def routing_table_for_query(query):
    print("query: " + query)
    return requests.get('http://pinot-broker.pinot:8099/debug/routingTable/sql', params={
        "query" : query
    }).json()

def routing_table_for_table(table):
    return requests.get(f'http://pinot-broker.pinot:8099/debug/routingTable/{table}', params={
        "query" : query
    }).json()

def external_view_for_table(table):
    return requests.get(f'http://pinot-controller.pinot:9000/tables/{table}/externalview').json()

def routing_table_for_query_dataframe(query):
    rt = routing_table_for_query(query)
    rt_data = {}

    for server, server_segments in rt.items():
        server_name = server_name_regex.search(server).group()
        for s in server_segments:
            rt_data[s] = server_name

    rt_data_list = []
    for segment, server in rt_data.items():
        rt_data_list.append({"segment": segment, "server": server})

    rt_data_list.sort(key=lambda L: (int(ordinal_pattern.search(L["segment"]).group(1)), L))
    return pd.DataFrame(rt_data_list)

def routing_table_for_table_dataframe(table):
    rt = routing_table_for_table(table)
    rt_data = {}

    for table_name_type, table_rt in rt.items():
        table_type = re.search('REALTIME|OFFLINE', table_name_type).group()
        for server, server_segments in table_rt.items():
            server_name = server_name_regex.search(server).group()
            for s in server_segments:
                try:
                    rt_data[s][table_type] = server_name
                except KeyError:
                    rt_data[s] = {table_type: server_name}

    rt_data_list = []
    for segment, type_server in rt_data.items():
        segment_data = {"segment": segment}
        for table_type, server in type_server.items():
            segment_data[table_type] = server
        rt_data_list.append(segment_data)

    rt_data_list.sort(key=lambda L: (int(ordinal_pattern.search(L["segment"]).group(1)), L))
    return pd.DataFrame(rt_data_list)

def external_view_for_table_dataframe(table):
    ev = external_view_for_table(table)
    ev_data = {}

    for table_type, ev_per_type in ev.items():
        if ev_per_type == None:
            continue
        
        for segment, segment_servers in ev_per_type.items():
            if not segment in ev_data:
                ev_data[segment] = {}
            for server, state in segment_servers.items():
                server_name = server_name_regex.search(server).group()
                try:
                    ev_data[segment][table_type].append(server_name)
                except KeyError:
                    ev_data[segment][table_type] = [server_name]

    return pd.DataFrame(ev_data).transpose()

In [391]:
external_view_for_table_dataframe("trips_segmentation_1")

,REALTIME
trips_segmentation_1__0__0__20210409T2006Z,[pinot-server-1]
trips_segmentation_1__0__10__20210409T2007Z,[pinot-server-1]
trips_segmentation_1__0__11__20210409T2007Z,[pinot-server-1]
trips_segmentation_1__0__12__20210409T2007Z,[pinot-server-1]
trips_segmentation_1__0__13__20210409T2007Z,[pinot-server-1]
trips_segmentation_1__0__14__20210409T2007Z,[pinot-server-1]
trips_segmentation_1__0__15__20210409T2007Z,[pinot-server-1]
trips_segmentation_1__0__16__20210409T2007Z,[pinot-server-1]
trips_segmentation_1__0__17__20210409T2007Z,[pinot-server-1]
trips_segmentation_1__0__18__20210409T2007Z,[pinot-server-1]


In [392]:
external_view_for_table_dataframe("trips_segmentation_2")

,REALTIME
trips_segmentation_2__0__0__20210409T2006Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__10__20210409T2007Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__1__20210409T2007Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__2__20210409T2007Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__3__20210409T2007Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__4__20210409T2007Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__5__20210409T2007Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__6__20210409T2007Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__7__20210409T2007Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"
trips_segmentation_2__0__8__20210409T2007Z,"[pinot-server-0, pinot-server-1, pinot-server-2]"


In [408]:
routing_table_for_query_dataframe(query_for_trips_segmentation_1.replace("trips_segmentation_1", "trips_segmentation_1_REALTIME"))

query: select count(*) from trips_segmentation_1_REALTIME where trip_start_time_millis < 1618028898841


,segment,server
0,trips_segmentation_1__0__0__20210409T2006Z,pinot-server-1
1,trips_segmentation_1__0__1__20210409T2007Z,pinot-server-1
2,trips_segmentation_1__0__2__20210409T2007Z,pinot-server-1
3,trips_segmentation_1__0__3__20210409T2007Z,pinot-server-1
4,trips_segmentation_1__0__4__20210409T2007Z,pinot-server-1
5,trips_segmentation_1__0__5__20210409T2007Z,pinot-server-1
6,trips_segmentation_1__0__6__20210409T2007Z,pinot-server-1
7,trips_segmentation_1__0__7__20210409T2007Z,pinot-server-1
8,trips_segmentation_1__0__8__20210409T2007Z,pinot-server-1
9,trips_segmentation_1__0__9__20210409T2007Z,pinot-server-1


In [409]:
routing_table_for_table_dataframe("trips_segmentation_1")

,segment,REALTIME
0,trips_segmentation_1__0__0__20210409T2006Z,pinot-server-1
1,trips_segmentation_1__0__1__20210409T2007Z,pinot-server-1
2,trips_segmentation_1__0__2__20210409T2007Z,pinot-server-1
3,trips_segmentation_1__0__3__20210409T2007Z,pinot-server-1
4,trips_segmentation_1__0__4__20210409T2007Z,pinot-server-1
5,trips_segmentation_1__0__5__20210409T2007Z,pinot-server-1
6,trips_segmentation_1__0__6__20210409T2007Z,pinot-server-1
7,trips_segmentation_1__0__7__20210409T2007Z,pinot-server-1
8,trips_segmentation_1__0__8__20210409T2007Z,pinot-server-1
9,trips_segmentation_1__0__9__20210409T2007Z,pinot-server-1


In [411]:
routing_table_for_query_dataframe(query_for_trips_segmentation_2.replace("trips_segmentation_2", "trips_segmentation_2_REALTIME"))

query: select count(*) from trips_segmentation_2_REALTIME where trip_start_time_millis < 1618028898841


,segment,server
0,trips_segmentation_2__0__0__20210409T2006Z,pinot-server-0
1,trips_segmentation_2__0__1__20210409T2007Z,pinot-server-1
2,trips_segmentation_2__0__2__20210409T2007Z,pinot-server-2
3,trips_segmentation_2__0__3__20210409T2007Z,pinot-server-0
4,trips_segmentation_2__0__4__20210409T2007Z,pinot-server-1
5,trips_segmentation_2__0__5__20210409T2007Z,pinot-server-2
6,trips_segmentation_2__0__6__20210409T2007Z,pinot-server-0
7,trips_segmentation_2__0__7__20210409T2007Z,pinot-server-1
8,trips_segmentation_2__0__8__20210409T2007Z,pinot-server-2
9,trips_segmentation_2__0__9__20210409T2007Z,pinot-server-0


In [412]:
routing_table_for_table_dataframe("trips_segmentation_2")

,segment,REALTIME
0,trips_segmentation_2__0__0__20210409T2006Z,pinot-server-1
1,trips_segmentation_2__0__1__20210409T2007Z,pinot-server-2
2,trips_segmentation_2__0__2__20210409T2007Z,pinot-server-0
3,trips_segmentation_2__0__3__20210409T2007Z,pinot-server-1
4,trips_segmentation_2__0__4__20210409T2007Z,pinot-server-2
5,trips_segmentation_2__0__5__20210409T2007Z,pinot-server-0
6,trips_segmentation_2__0__6__20210409T2007Z,pinot-server-1
7,trips_segmentation_2__0__7__20210409T2007Z,pinot-server-2
8,trips_segmentation_2__0__8__20210409T2007Z,pinot-server-0
9,trips_segmentation_2__0__9__20210409T2007Z,pinot-server-1
